# TimeRNNレイヤを実装する
TimeRNNレイヤは、RNNレイヤを時間方向に結合していくレイヤである

In [5]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのfunctions.py, layers.py, time_layers.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')


from common.time_layers import RNN
from common.functions import sigmoid

ローカル環境での実行です


### [演習]
* 以下のTimeRNNレイヤのクラスを完成させましょう

In [6]:
# ヒント

def test(a, b, c):
    print("a=%s"%a, "b=%s"%b, "c=%s"%c)
    return

params = [1,2,3]
test(*params) # *を変数前につけると、各引数に展開される

a=1 b=2 c=3


In [7]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        """
        Wx : 入力xにかかる重み
        Wh : １時刻前のhにかかる重み
        b : バイアス
        stateful : 中間層の出力を次のミニバッチ に渡す場合はTrueにする
        """
        # パラメータのリスト
        self.params = [Wx, Wh, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        
        self.layers = None
        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        """
        順伝播計算
        xs : 配列形状は、(バッチサイズ、時間数、前層のノード数)
        """
        Wx, Wh, b = self.params
        N, T, D = xs.shape # バッチサイズ(N)、時間数(Time)、前層のノード数(Data)
        D, H = Wx.shape # 入力層のノード数(Data)、中間層のノード数 (Hidden)
        # Dが上記とかぶっているが、結局同じに値になるので問題はない
        # 通常はalert関数を仕込んで、逆に違う値になる場合はエラーを吐くように実装する

        self.layers = []
        
        # hsは、中間層の出力hを時間方向につなげたもの(series)
        hs = np.empty((N, T, H))

        # 中間層の出力hを初期化する（statefulでない場合、もしくはhに何も値が入っていない場合）
        # 逆に、statefulである場合（stateful=true）、かつhに何か値が入っている場合は、初期化しない（値を引き継ぐ）
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H))

        # 時間方向に計算を進める
        for t in range(T):
            
            # RNNレイヤを定義する
            layer = RNN(*self.params) # *を変数前につけると、各引数に展開される
            
            # 時刻tのデータをRNNレイヤに入力する
            self.h = layer.forward(xs[:, t, :], self.h)
            
            # 中間層の出力hをhsに代入する
            hs[:, t, :] = self.h
            
            # レイヤを追加する
            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        """
        逆伝播計算
        dhs : 各時刻における出力層からの勾配を格納した変数. 配列形状は(バッチ数、時間数、中間層のノード数)
        """
        
        Wx, Wh, b = self.params
        N, T, H = dhs.shape # バッチサイズ、時間数、中間層のノード数
        D, H = Wx.shape # 前層のノード数、　中間層のノード数

        # dxsを初期化する. dxsは、各時刻におけるdxを格納する変数
        dxs = np.empty((N, T, D)) # バッチ数、時間数、前層のノード数
        
        # dhの初期値
        dh = 0
        
        # 勾配の初期値
        grads = [0, 0, 0] #Wxの勾配、 Whの勾配、 bの勾配
        
        # 時間方向と逆向きに計算を進める
        for t in reversed(range(T)):
            
            # RNNレイヤの呼び出し
            layer = self.layers[t]
            
            # RNNレイヤの逆伝播計算
            # RNNレイヤに入力される勾配は、2方向から来るので、2つの値を足す
            dx, dh = layer.backward(dhs[:, t, :] + dh) 

            # dxをdxsに格納する
            dxs[:, t, :] = dx

            # Wxの勾配、 Whの勾配、 bの勾配、をそれぞれ足し合わせる
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
        
        # Wxの勾配、 Whの勾配、 bの勾配、を保持しておく
        for i, grad in enumerate(grads):
            self.grads[i][:] = grad # 同じメモリ位置に代入
            
        # 最後の中間層のdhを保持しておく
        self.dh = dh

        return dxs

In [8]:
D = 1 # 前層のノード数
H = 5 # 中間層のノード数
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.zeros(H)

# オブジェクトの生成
time_rnn = TimeRNN(Wx, Wh, b)
print("id of time_rnn.grads[0]", id(time_rnn.grads[0]))
print()


id of time_rnn.grads[0] 140648368768944



In [10]:
# 順伝播計算
N = 4 # バッチサイズ
T = 5 # 時間数
xs = np.random.randn(N, T, D)
hs = time_rnn.forward(xs)
print("hs=", hs)
print()

hs= [[[ 0.58781201 -0.00528815  0.03039984  0.03720471  0.44903646]
  [ 0.01396011 -0.82290984  0.69254596  0.34451103 -0.93424384]
  [ 0.94549392  0.96779251  0.87032055 -0.37880956  0.98971682]
  [-0.87952054 -0.99560755  0.90158272  0.58194642 -0.99649668]
  [-0.8271489   0.99685491  0.60059039 -0.69544924  0.83840687]]

 [[-0.13953053  0.00110143 -0.00633355 -0.00775243 -0.10036294]
  [ 0.97831958  0.24622863 -0.09561192  0.04318699  0.96442708]
  [-0.73762023 -0.98752921  0.75237138  0.47214694 -0.99900929]
  [ 0.97284923  0.9958064   0.68456471 -0.48253154  0.99831269]
  [ 0.95990296 -0.99618482  0.89474323  0.74776916 -0.63951471]]

 [[-0.85142569  0.00989128 -0.05681936 -0.06951151 -0.71842206]
  [ 0.81691098  0.94289496 -0.84414755 -0.45539468  0.99804899]
  [ 0.89688233 -0.99641578 -0.58601649  0.62275128 -0.79579572]
  [ 0.99997894  0.64411006  0.66552282 -0.43137272  0.99915199]
  [ 0.71643688 -0.99141749  0.95221851  0.81246492 -0.95544214]]

 [[ 0.75618365 -0.0077421   0.

In [11]:

# 逆伝播計算
dhs = np.random.randn(N, T, H)
dxs = time_rnn.backward(dhs)
print("dxs=", dxs)
print()

print("id of time_rnn.grads[0]", id(time_rnn.grads[0]))
print()


dxs= [[[ 0.90883178]
  [-0.57511534]
  [ 0.02384305]
  [-0.04864235]
  [-0.78269488]]

 [[-5.45144289]
  [-0.21396392]
  [-0.42340142]
  [-0.11004912]
  [-1.11015582]]

 [[ 1.89651852]
  [ 0.5920902 ]
  [ 1.85475658]
  [ 0.19724699]
  [ 0.59254477]]

 [[-0.52854374]
  [-0.86790064]
  [-0.146587  ]
  [ 0.04372309]
  [ 0.53879591]]]

id of time_rnn.grads[0] 140648368768944

